In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import optuna

from torch.utils.data import DataLoader
from src.data import MovingMnistDataset
import torch.optim as optim
import time

from src.dl_models.unet import UNet
from src import model, visualization, evaluate, data, preprocessing,train


In [4]:
class Objective_optuna(object):
    def __init__(self, path, csv, device):
        # Called when the study is created
        self.device = device
        self.path = path
        self.csv = csv

    def __call__(self, trial):
        # Called on each trial

        # Hyperparameters
        n_channels=3
        n_classes=1

        # Load data
        mnist_dataset = MovingMnistDataset(path=self.path, csv=self.csv, shuffle=True)
        loader = DataLoader(mnist_dataset, batch_size=10)

        # Initialize network
        model = UNet(n_channels=n_channels, n_classes=n_classes, bilinear=True).to(device)
        model.train()

        # Hyperparameters
        lr = trial.suggest_float("lr", 1e-5, 1e-1)
        optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
        criterion_name =  trial.suggest_categorical("criterion", ["MSELoss", "L1Loss"])
        criterion = getattr(nn, criterion_name)()

        loss_history = train.train_model(
            model,
            loader,
            criterion,
            optimizer,
            self.device,
            curr_epoch=0,
            loss_history=[],
            train_for=2,
            verbose=True,
            checkpoint_every=0,
            print_cuda_mem=False
        )
        return loss_history[-1]

path = 'C:/Users/Ignacio/Desktop/moving_mnist/moving_mnist_train/'
csv = 'C:/Users/Ignacio/Desktop/moving_mnist/small_train.csv'

# Set device
USE_CUDA = True
device = torch.device('cuda' if torch.cuda.is_available()
                    and USE_CUDA else 'cpu')
print(f'Using {device}')

# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(sampler=optuna.samplers.TPESampler()) #default sampler
study.optimize(Objective_optuna(path, csv, device), n_trials=3)

print("\nBest trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-05-26 00:53:02,838] A new study created in memory with name: no-name-0b4849c7-9a29-4819-ac95-1b58d52b313a
Using cuda
EPOCH: 1 Time elapsed: 21.22 secs.
EPOCH: 2 [I 2021-05-26 00:53:46,396] Trial 0 finished with value: 0.008684334345161915 and parameters: {'lr': 0.03872757798331064, 'optimizer': 'Adam', 'criterion': 'MSELoss'}. Best is trial 0 with value: 0.008684334345161915.
Time elapsed: 20.40 secs.
EPOCH: 1 Time elapsed: 19.22 secs.
EPOCH: 2 [I 2021-05-26 00:54:24,888] Trial 1 finished with value: 0.03207194805145264 and parameters: {'lr': 0.07764137082545249, 'optimizer': 'RMSprop', 'criterion': 'L1Loss'}. Best is trial 0 with value: 0.008684334345161915.
Time elapsed: 19.10 secs.
EPOCH: 1 Time elapsed: 18.99 secs.
EPOCH: 2 [I 2021-05-26 00:55:03,285] Trial 2 finished with value: 0.010609702207148075 and parameters: {'lr': 0.028309843678441717, 'optimizer': 'RMSprop', 'criterion': 'MSELoss'}. Best is trial 0 with value: 0.008684334345161915.
Time elapsed: 19.24 secs.

Best 